<a href="https://colab.research.google.com/github/Fatima-jawad/youtube-regression-project/blob/main/scraping_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install beautifulsoup4
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time, os
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("") 

InvalidArgumentException: ignored

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_best-selling_mobile_phones#Top-selling_mobile_phones"
wikipage = requests.get(url)
print(wikipage.status_code)


200


In [ ]:
soup = BeautifulSoup(wikipage.text, "html5lib")

In [ ]:
print(soup.prettify()[:100])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <


In [ ]:
columns = soup.find('table', class_='wikitable').find('tr')#.find_all('th') 
type(columns)

bs4.element.Tag

In [ ]:
print(columns.prettify())
print(columns.text)

<tr>
 <th>
  Manufacturer
 </th>
 <th>
  Model
 </th>
 <th>
  Form factor
 </th>
 <th>
  Smartphone?
 </th>
 <th>
  Year
 </th>
 <th colspan="2" data-sort-type="number">
  Graph (million units)
 </th>
</tr>


Manufacturer

Model

Form factor

Smartphone?

Year

Graph (million units)



In [ ]:
column_names = columns.text

In [ ]:
column_names = column_names.replace('\n','  ').strip().split('  ')

In [ ]:
column_names = [value for value in column_names if value != '']

In [ ]:
column_names.append('dropthis')

In [ ]:
data = soup.find('table', class_='wikitable').find('tbody')

In [ ]:
# print(data.prettify())
type(data)

bs4.element.Tag

In [ ]:
devices_list = []
for row in data.find_all('tr')[1:]:
  devices_list.append(row.text.strip().split('\n\n'))

In [ ]:
devices_list

[['Nokia', '1100', 'Bar', 'No', '2003', '250[3][1][4]', '250'],
 ['Nokia', '1110', 'Bar', 'No', '2005', '250[3][1][5]', '250'],
 ['Apple',
  'iPhone 6 and iPhone 6 Plus',
  'Touchscreen',
  'Yes',
  '2014',
  '222[2][3]',
  '222'],
 ['Nokia', '105 (2013) and 105 (2015)', 'Bar', 'No', '2013', '200[6]', '200'],
 ['Nokia', '3210', 'Bar', 'No', '1999', '160[1][5][7]', '160'],
 ['Nokia', '6600', 'Bar', 'Yes', '2003', '150[3][1]', '150'],
 ['Nokia', '1200', 'Bar', 'No', '2007', '150[3][1][5]', '150'],
 ['Nokia', '5230', 'Touchscreen', 'Yes', '2009', '150[3][1][5]', '150'],
 ['Samsung', 'E1100', 'Bar', 'No', '2009', '150[3]', '150'],
 ['Nokia',
  '2600 (2610/2626/2630)',
  'Bar',
  'No',
  '2004',
  '135[3][1][5][4]',
  '135'],
 ['Motorola', 'RAZR V3', 'Flip phone', 'No', '2004', '130[3][5]', '130'],
 ['Nokia', '1600 (1650/1661)', 'Bar', 'No', '2005', '130[3][1][5][4]', '130'],
 ['Nokia', '3310 (3330)', 'Bar', 'No', '2000', '126[3][1][5][8]', '126'],
 ['Apple', 'iPhone 11', 'Touchscreen', 'Ye

In [ ]:
df = pd.DataFrame(devices_list, columns=column_names)
df.head()

,Manufacturer,Model,Form factor,Smartphone?,Year,Graph (million units),dropthis
0,Nokia,1100,Bar,No,2003,250[3][1][4],250
1,Nokia,1110,Bar,No,2005,250[3][1][5],250
2,Apple,iPhone 6 and iPhone 6 Plus,Touchscreen,Yes,2014,222[2][3],222
3,Nokia,105 (2013) and 105 (2015),Bar,No,2013,200[6],200
4,Nokia,3210,Bar,No,1999,160[1][5][7],160


In [ ]:
df.columns

Index(['Manufacturer', 'Model', 'Form factor', 'Smartphone?', 'Year',
       'Graph (million units)', 'dropthis'],
      dtype='object')

In [ ]:
df.drop(columns=['Graph (million units)'],inplace=True)
df.head()

,Manufacturer,Model,Form factor,Smartphone?,Year,dropthis
0,Nokia,1100,Bar,No,2003,250
1,Nokia,1110,Bar,No,2005,250
2,Apple,iPhone 6 and iPhone 6 Plus,Touchscreen,Yes,2014,222
3,Nokia,105 (2013) and 105 (2015),Bar,No,2013,200
4,Nokia,3210,Bar,No,1999,160


In [ ]:
df.rename(columns={'dropthis': 'million units'})

,Manufacturer,Model,Form factor,Smartphone?,Year,million units
0,Nokia,1100,Bar,No,2003,250
1,Nokia,1110,Bar,No,2005,250
2,Apple,iPhone 6 and iPhone 6 Plus,Touchscreen,Yes,2014,222
3,Nokia,105 (2013) and 105 (2015),Bar,No,2013,200
4,Nokia,3210,Bar,No,1999,160
...,...,...,...,...,...,...
110,LG,G2,Touchscreen,Yes,2013,3
111,LeTV,LeEco Le 1s,Touchscreen,Yes,2016,3
112,Google,Pixel and Pixel XL,Touchscreen,Yes,2016,2.1
113,Palm,Centro,Keyboard bar,Yes,2007,2
